In [ ]:
import torch
import numpy as np
import plotly.graph_objects as go
import plotly.express as px

from nemo.util import parse_colmap_point_cloud
from nemo.plotting import plot_3d_points

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

%load_ext autoreload
%autoreload 2

# Red Rocks

### COLMAP sparse point cloud

In [ ]:
path = '../data/redrocks_points3D.txt'
points, colors = parse_colmap_point_cloud(path)

In [ ]:
# Plotting the points
fig = go.Figure()
fig = plot_3d_points(fig, x=points[:,0], y=points[:,1], z=points[:,2])
fig.show()

In [ ]:
# Extract the points within range xyz in [-5, 5]
points = points[(points[:,0] > -5) & (points[:,0] < 5)]
points = points[(points[:,1] > -5) & (points[:,1] < 5)]
points = points[(points[:,2] > -5) & (points[:,2] < 5)]

fig = go.Figure()
fig = plot_3d_points(fig, x=points[:,0], y=points[:,1], z=points[:,2], markersize=1)
fig.show()

In [ ]:
# Estimate the ground plane
from nemo import geometry
plane = geometry.fit_plane(points)
plane

# We should use the nerfstudio dataparser